[View in Colaboratory](https://colab.research.google.com/github/kakods/FashionMNIST/blob/master/CNN.ipynb)

In [0]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
train_dataset=dsets.FashionMNIST(root='./data',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)
test_dataset=dsets.FashionMNIST(root='./data',
                        train=False,
                        transform=transforms.ToTensor())

In [0]:
batch_size=100
n_iters=3000
num_epochs=n_iters/(len(train_dataset)/batch_size)
num_epochs=int(num_epochs)

train_loader=torch.utils.data.DataLoader(dataset=train_dataset,
                                   batch_size=batch_size,
                                   shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,
                                  batch_size=batch_size,
                                  shuffle=False)

In [0]:
class CNNModel(nn.Module):
  def __init__(self):
    super(CNNModel,self).__init__()
    
    self.cnn1=nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2)
    self.relu1=nn.ReLU()
    
    self.maxpool1=nn.MaxPool2d(kernel_size=2)
    
    self.cnn2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2)
    self.relu2=nn.ReLU()
    
    self.maxpool2=nn.MaxPool2d(kernel_size=2)
    self.fc1=nn.Linear(32*7*7,10)
    
  def forward(self,x):
    out=self.cnn1(x)
    out=self.relu1(out)
    out=self.maxpool1(out)
    out=self.cnn2(out)
    out=self.relu2(out)
    out=self.maxpool2(out)
    out=out.view(out.size(0),-1)
    out=self.fc1(out)
    return out
  

In [0]:
model=CNNModel()
if torch.cuda.is_available():
  model.cuda()


In [0]:
criterion=nn.CrossEntropyLoss()

In [0]:
learning_rate=0.1
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

In [47]:
iter=0
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
          images=Variable(images.cuda())
          labels=Variable(labels.cuda())
        else:
          images=Variable(images)
          labels=Variable(labels)
        
        optimizer.zero_grad()
        
        outputs=model(images)
        
        loss=criterion(outputs,labels)
        
        loss.backward()
        
        optimizer.step()
        
        iter+=1
        
        if iter%500==0:
            correct=0
            total=0
            for images,labels in test_loader:
                if torch.cuda.is_available():
                  images=Variable(images.cuda())
                else:
                  images=Variable(images)
                outputs=model(images)
                _, predicted=torch.max(outputs.data,1)
                total+=labels.size(0)
                
                if torch.cuda.is_available():
                  correct+=(predicted.cpu()==labels.cpu()).sum()
                else:
                  correct+=(predicted==labels).sum()
            accuracy=100*correct/total
            print('Iteration: {}, Loss: {}, Accuracy: {}'.format(iter,loss.data[0],accuracy))

Iteration: 500, Loss: 0.45950016379356384, Accuracy: 81.97
Iteration: 1000, Loss: 0.5141852498054504, Accuracy: 85.93
Iteration: 1500, Loss: 0.23450490832328796, Accuracy: 87.39
Iteration: 2000, Loss: 0.23029525578022003, Accuracy: 87.59
Iteration: 2500, Loss: 0.13793593645095825, Accuracy: 88.31
Iteration: 3000, Loss: 0.3300165832042694, Accuracy: 88.86
